In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from sklearn.linear_model import RidgeCV
from sklearn.isotonic import IsotonicRegression

In [ ]:
from pyseir import load_data
from pyseir.inference.infer_t0 import infer_t0
from pyseir.models.suppression_policies import distancing_measure_suppression

In [ ]:
import us
from pyseir.inference.infer_rt import RtInferenceEngine
engine = RtInferenceEngine(fips='06')
df = engine.infer_all(plot=False, shift_deaths=0)

In [ ]:
whitelist = load_data.load_whitelist()
whitelist_fips = whitelist[whitelist["inference_ok"]==True]["fips"]

In [ ]:
df_npi = load_data.load_public_implementations_data()
df_npi_whitelisted = df_npi.loc[whitelist_fips]

In [ ]:
def make_intervention_design(row):    
    ts = pd.Series({(v, k): 1 for k, v in row.iteritems()})
    return ts.unstack(level=1).fillna(method='ffill').fillna(0)

npi_ts = {idx: make_intervention_design(row) for idx, row in df_npi_whitelisted.iterrows()}

In [ ]:
%%time
intervention_ts_accum = []
interventions = ['500_gatherings', '50_gatherings', 'entertainment_gym',
       'federal_guidelines', 'foreign_travel_ban', 'public_schools',
       'restaurant_dine-in', 'stay_at_home']
for k, npi in npi_ts.items():
    try:
        engine = RtInferenceEngine(fips=k)
        r_t = engine.infer_all(plot=False, shift_deaths=0)
    except KeyError:
        print(f"{k} not inferred")
        continue
    fips_design = pd.merge(npi, df["Rt_MAP__new_cases"], 
                           how='outer', right_index=True, left_index=True)\
                      .fillna(method='ffill').fillna(0)
    fips_design[interventions] = fips_design[interventions].cumsum()
    fips_design["fips"] = k
    fips_design.set_index("fips", append=True, inplace=True)
    intervention_ts_accum.append(fips_design)


In [ ]:
intervention_ts = pd.concat(intervention_ts_accum)

In [ ]:
intervention_ts.index = intervention_ts.index.rename(["date", "fips"])
intervention_ts.index = intervention_ts.index.reorder_levels(["fips", "date"])

In [ ]:
%%time
ridge_regressors = {}
nonneg_regressors = {}
R0 = intervention_ts["Rt_MAP__new_cases"].max()
intervention_ts["Rt_percent_decrease"] = (R0 - intervention_ts["Rt_MAP__new_cases"]) / R0

for lag in range(15):
    df_accum = []
    for fips, df in intervention_ts.groupby("fips"):
        df["Rt_percent_decrease"] = pd.Series(df["Rt_percent_decrease"].values[lag:], 
                                              index=df.index.values[:-lag if lag else None])
        df_accum.append(df.dropna())
    lagged_df = pd.concat(df_accum) 
    X = lagged_df[interventions].applymap(lambda x: 1 if x > 0 else 0)
    y = lagged_df["Rt_percent_decrease"]
    ridge = RidgeCV()
    ridge_regressors[lag] = {"model": ridge.fit(X, y), "score": ridge.score(X, y)}
    for i, c in zip(interventions, ridge.coef_):
        ridge_regressors[lag][i] = c
    ridge_regressors[lag]["intercept"] = ridge.intercept_
    X_with_intercept = np.hstack((X, np.ones(X.shape[0]).reshape(-1, 1)))
    nonneg_coef, residual = nnls(X_with_intercept, y)
    nonneg_regressors[lag] = {"residual": residual}
    for i, c in zip(interventions + ["intercept"], nonneg_coef):
        nonneg_regressors[lag][i] = c

In [ ]:
ridge_results = pd.DataFrame(ridge_regressors).T
ridge_results[interventions + ["intercept", "score"]]

In [ ]:
nonneg_results = pd.DataFrame(nonneg_regressors).T
nonneg_results[interventions + ["intercept", "residual"]]